In [1]:
from smadata import getSMAData

import pandas as pd
import numpy as np

def preprocess_SMA(xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs):
    # Combine all the training and testing data for normalization
    all_train_data = pd.concat(xtrain_dfs.values())
    all_test_data = pd.concat(xtest_dfs.values())

    # Normalize the data using the combined dataset mean and std
    mean = np.nanmean(all_train_data)
    std = np.nanstd(all_train_data)

    # Preprocess each inverter's data
    train_data_list = []
    train_labels_list = []
    test_data_list = []
    test_labels_list = []

    for inv in xtrain_dfs.keys():
        xtrain = xtrain_dfs[inv].values.astype(np.float64)
        xtest = xtest_dfs[inv].values.astype(np.float64)
        ytrain = ytrain_dfs[inv].values.flatten()
        ytest = ytest_dfs[inv].values.flatten()

        # Normalize the data
        xtrain = (xtrain - mean) / std
        xtest = (xtest - mean) / std

        # Reshape the data to add a new axis
        train_data_list.append(xtrain[..., np.newaxis])
        train_labels_list.append(ytrain)
        test_data_list.append(xtest[..., np.newaxis])
        test_labels_list.append(ytest)

    # Concatenate all inverter data into single arrays
    train_data = np.concatenate(train_data_list, axis=0)
    train_labels = np.concatenate(train_labels_list, axis=0)
    test_data = np.concatenate(test_data_list, axis=0)
    test_labels = np.concatenate(test_labels_list, axis=0)

    return train_data, train_labels, test_data, test_labels

# Usage
xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs = getSMAData()
train_data, train_labels, test_data, test_labels = preprocess_SMA(xtrain_dfs, xtest_dfs, ytrain_dfs, ytest_dfs)


(6057, 137) (11645, 137)
(6062, 137) (11231, 137)
(6180, 137) (11258, 137)
(5490, 137) (11139, 137)
(5516, 137) (10925, 137)
(5980, 137) (12281, 137)
(5923, 137) (10865, 137)
(6180, 137) (12319, 137)
(6180, 137) (8601, 137)
(6180, 137) (12146, 137)
(6165, 137) (11853, 137)
(6176, 137) (11752, 137)
(5741, 137) (12236, 137)
(6180, 137) (12122, 137)
(6159, 137) (12339, 137)
(6162, 137) (12333, 137)
(6180, 137) (12340, 137)
(6180, 137) (12339, 137)
(6180, 137) (12310, 137)


In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from daemon.model import DAEMON

# Assuming train_data, train_labels, test_data, test_labels have been preprocessed
def create_dataloader(data, labels, batch_size=32):
    # Create a TensorDataset and DataLoader
    dataset = TensorDataset(torch.FloatTensor(data), torch.FloatTensor(labels))
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Prepare the data
train_dataloader = create_dataloader(train_data, train_labels)
test_dataloader = create_dataloader(test_data, test_labels)

# Define model options
class Opt:
    ngpu = 1
    ndf = 64  # Example value
    nz = 32   # Example value
    lr_d = 0.0002
    lr_g = 0.0002
    beta1 = 0.5
    train_batchsize = 32
    val_batchsize = 32
    test_batchsize = 32
    niter = 100  # Adjust as needed
    patience = 10
    w_rec = 1
    w_lat = 1
    print_freq = 10

opt = Opt()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the DAEMON model
daemon_model = DAEMON(opt, train_dataloader, None, test_data, test_labels, device)

# Forward pass through the model
def forward_pass(model, dataloader):
    model.G.eval()  # Set the model to evaluation mode
    all_outputs = []

    with torch.no_grad():
        for data, _ in dataloader:
            data = data.permute([0, 2, 1]).to(device)  # Reshape as needed
            outputs, _, _, _ = model.G(data)  # Forward pass
            all_outputs.append(outputs.cpu().numpy())  # Store outputs

    return np.concatenate(all_outputs, axis=0)

# Perform the forward pass on the test data
outputs = forward_pass(daemon_model, test_dataloader)

# outputs now contains the results from the generator model
print("Outputs from the DAEMON model:", outputs)


AttributeError: 'Opt' object has no attribute 'dataset'